In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow import keras

In [78]:
data = pd.read_csv(r"Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [79]:
x = data.drop(columns=['RowNumber' , 'CustomerId' , 'Exited'])
y = data['Exited']

In [80]:
x.head()

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [113]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Step 1: FRESH data load karo (re-read the CSV)
df = pd.read_csv('Churn_Modelling.csv')  # ← Original file path

# Step 2: Check original data
print("Original data types:")
print(df[['Geography', 'Gender']].dtypes)
print("\nSample data:")
print(df[['Geography', 'Gender']].head())

# Step 3: Create x (DON'T use previously encoded x)
x = df.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1, errors='ignore')
y = df['Exited']

# Step 4: Check object columns
object_cols = x.select_dtypes(include=['object']).columns.tolist()
print(f"\nObject columns found: {object_cols}")

# Step 5: Encode
LabelEncoder_dict1 = {}

for col in object_cols:
    le = LabelEncoder()
    print(f"\nEncoding {col}: {x[col].unique()}")
    x[col] = le.fit_transform(x[col])
    LabelEncoder_dict1[col] = le
    print(f"  ✓ Saved classes: {le.classes_}")

# Step 6: Save
with open('LabelEncoder_file1.pkl', 'wb') as file:
    pickle.dump(LabelEncoder_dict1, file)

print(f"\n✓✓✓ Total encoders saved: {len(LabelEncoder_dict1)}")

# Step 7: Verify immediately
with open('LabelEncoder_file1.pkl', 'rb') as file:
    test = pickle.load(file)
    print("\n=== VERIFICATION ===")
    for col, encoder in test.items():
        print(f"{col}: {encoder.classes_}")

Original data types:
Geography    object
Gender       object
dtype: object

Sample data:
  Geography  Gender
0    France  Female
1     Spain  Female
2    France  Female
3    France  Female
4     Spain  Female

Object columns found: ['Geography', 'Gender']

Encoding Geography: ['France' 'Spain' 'Germany']
  ✓ Saved classes: ['France' 'Germany' 'Spain']

Encoding Gender: ['Female' 'Male']
  ✓ Saved classes: ['Female' 'Male']

✓✓✓ Total encoders saved: 2

=== VERIFICATION ===
Geography: ['France' 'Germany' 'Spain']
Gender: ['Female' 'Male']


In [114]:
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [115]:
x_train , x_test , y_train , y_test = train_test_split(x,y,random_state=2 , test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [116]:
with open('scaler.pkl' , 'wb') as file:
    pickle.dump(ss , file)

ANN Implementation

In [117]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU, Activation
from tensorflow.keras import optimizers, regularizers
import tensorflow as tf

def build_model(hp):
    model = Sequential()

    # -----------------------------
    # Tune batch_size and epochs HERE
    # -----------------------------
    # These need to be defined in build_model to be tracked
    hp.Int('batch_size', 16, 128, step=16, default=32)
    hp.Int('epochs', 10, 100, step=10, default=50)

    # -----------------------------
    # Tune number of hidden layers
    # -----------------------------
    num_layers = hp.Int("num_layers", 1, 5, step=1)
    for i in range(num_layers):
        units = hp.Int(f"units_{i}", 16, 256, step=16)
        activation = hp.Choice(
            f"activation_{i}",
            [
                "sigmoid", "tanh", "relu", "leaky_relu", "prelu", "elu",
                "selu", "swish", "softmax", "gelu", "mish", "hard_swish",
                "hard_sigmoid", "linear", "binary_step"
            ]
        )
        l2_reg = hp.Float(f"l2_{i}", 0.0, 0.01, step=0.001)

        model.add(Dense(units=units, kernel_regularizer=regularizers.l2(l2_reg) if l2_reg > 0 else None))

        # Apply activation
        if activation == "leaky_relu":
            model.add(LeakyReLU())
        elif activation == "prelu":
            model.add(PReLU())
        elif activation == "elu":
            model.add(ELU())
        elif activation == "swish":
            model.add(Activation(tf.nn.swish))
        elif activation == "gelu":
            model.add(Activation(tf.nn.gelu))
        elif activation == "mish":
            model.add(Activation(lambda x: x * tf.math.tanh(tf.math.softplus(x))))
        elif activation == "hard_swish":
            model.add(Activation(tf.nn.hard_swish))
        elif activation == "binary_step":
            model.add(Activation(lambda x: tf.where(x >= 0, 1.0, 0.0)))
        else:
            model.add(Activation(activation))

        dropout_rate = hp.Float(f"dropout_{i}", 0.0, 0.5, step=0.05)
        if dropout_rate > 0:
            model.add(Dropout(rate=dropout_rate))

    # Output layer
    model.add(Dense(1, activation="sigmoid"))

    # -----------------------------
    # Tune optimizer
    # -----------------------------
    optimizer_name = hp.Choice(
        "optimizer",
        ["sgd", "momentum", "nesterov", "adagrad", "adadelta",
         "rmsprop", "adam", "adamax", "nadam", "adamw"]
    )
    lr = hp.Float("learning_rate", 1e-5, 1e-2, sampling="log")

    if optimizer_name == "sgd":
        optimizer = optimizers.SGD(learning_rate=lr)
    elif optimizer_name == "momentum":
        optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9)
    elif optimizer_name == "nesterov":
        optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    elif optimizer_name == "adagrad":
        optimizer = optimizers.Adagrad(learning_rate=lr)
    elif optimizer_name == "adadelta":
        optimizer = optimizers.Adadelta(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = optimizers.RMSprop(learning_rate=lr)
    elif optimizer_name == "adam":
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "adamax":
        optimizer = optimizers.Adamax(learning_rate=lr)
    elif optimizer_name == "nadam":
        optimizer = optimizers.Nadam(learning_rate=lr)
    elif optimizer_name == "adamw":
        optimizer = optimizers.AdamW(learning_rate=lr)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Custom tuner class to use batch_size and epochs from hyperparameters
class MyTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        # Extract batch_size and epochs from the trial's hyperparameters
        kwargs['batch_size'] = trial.hyperparameters.get('batch_size')
        kwargs['epochs'] = trial.hyperparameters.get('epochs')
        return super(MyTuner, self).run_trial(trial, *args, **kwargs)

# -----------------------------
# Initialize tuner
# -----------------------------
tuner = MyTuner(
    build_model,
    objective="val_accuracy",
    max_trials=25,
    directory="my_tuner",
    project_name="full_model_tuning"
)

# -----------------------------
# Run hyperparameter search
# -----------------------------
tuner.search(
    x_train,
    y_train,
    validation_data=(x_test, y_test)
)

# -----------------------------
# Print best hyperparameters
# -----------------------------
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\nBest Hyperparameters:")
print("---------------------")
print("Optimizer:", best_hp.get("optimizer"))
print("Learning Rate:", best_hp.get("learning_rate"))
print("Number of Layers:", best_hp.get("num_layers"))

for i in range(best_hp.get("num_layers")):
    print(f"\nLayer {i}:")
    print("  Units:", best_hp.get(f"units_{i}"))
    print("  Activation:", best_hp.get(f"activation_{i}"))
    print("  Dropout:", best_hp.get(f"dropout_{i}"))
    print("  L2:", best_hp.get(f"l2_{i}"))

print("\nTraining Configuration:")
# Handle case where batch_size and epochs might not exist in saved tuner
try:
    print("Batch Size:", best_hp.get("batch_size"))
except KeyError:
    print("Batch Size: Not tuned (using default)")

try:
    print("Epochs:", best_hp.get("epochs"))
except KeyError:
    print("Epochs: Not tuned (using default)")

Reloading Tuner from my_tuner\full_model_tuning\tuner0.json

Best Hyperparameters:
---------------------
Optimizer: adamw
Learning Rate: 0.0017336359685746886
Number of Layers: 2

Layer 0:
  Units: 128
  Activation: hard_sigmoid
  Dropout: 0.05
  L2: 0.0

Layer 1:
  Units: 64
  Activation: elu
  Dropout: 0.05
  L2: 0.007

Training Configuration:
Batch Size: 32
Epochs: 100


In [124]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Best hyperparameters from tuner
learning_rate = 0.00020770137026701493

# Define optimizer
optimizer = Adadelta(learning_rate=learning_rate)

# Build model
model = Sequential()
model.add(Input(shape=(10,)))  # input layer with 8 features

# Layer 0
model.add(Dense(112,  kernel_regularizer=l2(0.009000000000000001)))
model.add(PReLU())   
model.add(Dropout(0.0))

# Layer 1
model.add(Dense(256,  kernel_regularizer=l2(0.003)))
model.add(PReLU())   
model.add(Dropout(0.4))
# Layer 2
model.add(Dense(224, activation='elu', kernel_regularizer=l2(0.004)))
model.add(Dropout(0.05))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(optimizer=optimizer, 
              loss='mean_squared_error', 
              metrics=['accuracy'])

# set up the TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Summary 
model.summary()



from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',      # or 'val_accuracy' if you prefer
    patience=20,             # wait for 20 epochs with no improvement
    restore_best_weights=True, # restore the best weights after stopping
    verbose=1
)
# Tensor Board callback
tenserflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
model.fit(
    x_train, 
    y_train, 
    validation_data=(x_test, y_test),
    epochs=90,              # maximum epochs
    batch_size=80,            # or the best batch size from tuner
   callbacks=[early_stop , tenserflow_callback]
)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                │ (None, 112)            │         1,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_20 (PReLU)              │ (None, 112)            │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 256)            │        28,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_21 (PReLU)              │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 224)            │        57,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 1)              │           225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,321 (345.00 KB)

 Trainable params: 88,321 (345.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5721 - loss: 1.8374 - val_accuracy: 0.5995 - val_loss: 1.8363
Epoch 2/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5771 - loss: 1.8357 - val_accuracy: 0.6170 - val_loss: 1.8342
Epoch 3/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6068 - loss: 1.8323 - val_accuracy: 0.6355 - val_loss: 1.8321
Epoch 4/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6087 - loss: 1.8314 - val_accuracy: 0.6520 - val_loss: 1.8299
Epoch 5/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6109 - loss: 1.8300 - val_accuracy: 0.6720 - val_loss: 1.8277
Epoch 6/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6324 - loss: 1.8272 - val_accuracy: 0.6845 - val_loss: 1.8256
Epoch 7/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6419 - loss: 1.8259 - val_accuracy: 0.7055 - val_loss: 1.8234
Epoch 8/90
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6494 - loss: 1.8229 - val_accuracy: 0

In [119]:
model.save('model.h5')

In [120]:
#let load TenserBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [121]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit/20251209-154539

Reusing TensorBoard on port 6012 (pid 13252), started 0:37:24 ago. (Use '!kill 13252' to kill it.)

In [122]:
# Load The Pickel Files